In [ ]:
!pip install kaleido
!pip install cohere
!pip install openai
!pip install tiktoken

!pip install -q -U gradio
!pip install -q -U TTS
!pip install -q -U faster-whisper
!pip install -q -U moviepy
!pip install -q -U deepl
!pip install -q -U librosa
!pip install -q numpy==1.22.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 79.9/79.9 MB 44.4 MB/s eta 0:00:01
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 32.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (m

In [1]:
import os
import moviepy.editor as mp
from faster_whisper import WhisperModel
import deepl
import torch
from TTS.api import TTS
import librosa
import gradio as gr

In [ ]:
# Define function to extract audio from video
def extract_audio(video_path):
    clip = mp.VideoFileClip(video_path)
    audio_path = os.path.splitext(video_path)[0] + ".wav"
    clip.audio.write_audiofile(audio_path)
    return audio_path

# Define function to perform speech diarization
def speech_diarization(audio_path):
    diarization = whisperx.Diarization(audio_path)
    return diarization.transcription

# Define function to perform text translation
def text_translation(text, target_language):
    translation = deepl.translate(text, target_language)
    return translation

# Define function to perform voice cloning
def voice_cloning(text, speaker_model_path):
    tts = coqui_tts.TTS(speaker_model_path)
    audio = tts.get_audio(text)
    return audio

# Define function to adjust voice pace
def adjust_voice_pace(audio_path, target_duration):
    y, sr = librosa.load(audio_path, sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    tempo = duration / target_duration
    y_stretched = librosa.effects.time_stretch(y, tempo)
    librosa.output.write_wav(audio_path, y_stretched, sr)
    return audio_path

# Define function to perform video translation
def video_translation(video_path, target_language, speaker_model_path):
    # Extract audio from video
    audio_path = extract_audio(video_path)
    # Perform speech diarization
    transcription = speech_diarization(audio_path)
    # Perform text translation
    translated_transcription = []
    for text in transcription:
        translated_text = text_translation(text, target_language)
        translated_transcription.append(translated_text)
    # Perform voice cloning
    cloned_audio_paths = []
    for i, text in enumerate(translated_transcription):
        speaker_model_path_i = speaker_model_path[i] if len(speaker_model_path) > i else speaker_model_path[-1]
        cloned_audio_path = voice_cloning(text, speaker_model_path_i)
        cloned_audio_paths.append(cloned_audio_path)
    # Adjust voice pace
    target_duration = mp.VideoFileClip(video_path).duration
    for i, audio_path in enumerate(cloned_audio_paths):
        cloned_audio_paths[i] = adjust_voice_pace(audio_path, target_duration)
    # Combine video with new audio
    video = mp.VideoFileClip(video_path)
    audio_clips = [mp.AudioFileClip(audio_path) for audio_path in cloned_audio_paths]
    audio = mp.concatenate_audioclips(audio_clips)
    video_with_new_audio = video.set_audio(audio)
    # Save video with new audio
    video_with_new_audio_path = os.path.splitext(video_path)[0] + "_" + target_language + ".mp4"
    video_with_new_audio.write_videofile(video_with_new_audio_path)
    return video_with_new_audio_path


In [ ]:
def translate_video(video_path, target_language, speaker_model_path):
    video_with_new_audio_path = video_translation(video_path, target_language, speaker_model_path)
    return gr.outputs.Video(video_with_new_audio_path)


inputs = [
    gr.inputs.Video(type="file", label="Select a video file"),
    gr.inputs.Dropdown(["uk", "ru"], label="Select target language"),
    gr.inputs.Text(label="Speaker model path (optional)", default="")
]

outputs = gr.outputs.Video(label="Translated video")
gr.Interface(fn=translate_video, inputs=inputs, outputs=outputs, title="YouTube Video Translation Program").launch()

